In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
header = {
    "Authority":"www.moneycontrol.com",
    "Method":"GET",
    "Accept-Encoding":"gzip, deflate, br",
    "Accept-Language":"en-US,en;q=0.9",
}

In [36]:
def soup_to_balanceSheet(bssoup,default_rows=[]):
    tables = bssoup.find_all("table")
    balanceSheet = {}
    try:
        table_of_bs = tables[1]
    except:
        return balanceSheet     
    table_trs = table_of_bs.find_all("tr")
    for tr in table_trs[3:]:
        cols = tr.find_all("td")
        if cols[1].text != "":
            balanceSheet[cols[0].text.upper()] = float(cols[1].text.replace(",","").replace("--","0"))
    return balanceSheet

def coy_to_BSlink(coy_anchor,symbol="",linkquate=""):
    link = "https://www.moneycontrol.com/financials/"+symbol+"/balance-sheetVI/"+linkquate+"#"+linkquate
    try:
        temp_url = "https://www.moneycontrol.com/"+coy_anchor.attrs["href"]
    except:
        return link
    coy_content_html  = requests.get(temp_url).text
    coy_soup = bs(coy_content_html)
    for box in coy_soup.findAll("div",{"class":"qlnkBx"}):
        if(box.find("h4").text=="Financials"):
            anchor = box.findAll("a",{"title":"Balance Sheet"})[0]
            try:
                link = anchor.attrs["href"]
            except:
                return link    
    return link

def catagory_BalanceSheet(sector_link):
    sector_name = sector_link.split("/")[-1].replace(".html","")
    print("fetching companies in sector : "+sector_name)
    cat_content = requests.get(sector_link,headers = header)
    cat_soup = bs(cat_content.text)
    # try to fetch companies name for the related sector
    try:
        cat_anchor_list = cat_soup.find_all("table")[2].find_all("a")
        print(len(cat_anchor_list))
        if len(cat_anchor_list) <= 0:
            print("sorry companies link is not present")
            return ({},False)
    except:
        print("sorry no. company found in sector : "+sector_name)
        return ({},False,sector_name)
    raw_data =[]
    for a in cat_anchor_list:
        row = {}
        inside_a = a.attrs['href'].split("/")
        row['name'] = a.text
        row['catagory'] = inside_a[3]
        row['symbol'] = inside_a[4].upper()
        row['linkQuate'] = inside_a[5]
        row['BS_standalone_link'] = coy_to_BSlink(a,inside_a[4],inside_a[5]) 
        raw_data.append(row)
    dataframe = pd.DataFrame(raw_data)
    raw_balanceSheets = []
    for link in dataframe['BS_standalone_link']:
        if(link!=None):
            content_html = requests.get(link,headers=header)
            balance_sheet_soup = bs(content_html.text)
            raw_balanceSheets.append(soup_to_balanceSheet(balance_sheet_soup))
        else:
            raw_balanceSheets.append({})
    dataframe_balanceSheet = pd.DataFrame(raw_balanceSheets)   
    full_cat_balaceSheet = pd.concat([dataframe,dataframe_balanceSheet],axis=1)
    return (full_cat_balaceSheet,True,sector_name)

In [42]:
big_data = {}

In [43]:
prev_batch = 0 
for batch in range(5,50,5):
    print(prev_batch,"- ",batch)

    for link in cat_links[prev_batch:batch]:
        sector  = catagory_BalanceSheet(link)
        if sector[1]:
    #         big_data.append((sector[0]))
            big_data[sector[2]] = sector[0]
    prev_batch = batch


0 -  5
fetching companies in sector : abrasives
4
fetching companies in sector : aluminium
5
fetching companies in sector : aquaculture
2
fetching companies in sector : auto-2-3-wheelers
6
fetching companies in sector : auto-cars-jeeps
3
5 -  10
fetching companies in sector : auto-lcvshcvs
0
sorry companies link is not present
fetching companies in sector : auto-tractors
3
fetching companies in sector : auto-ancillaries
57
fetching companies in sector : banks-private-sector
19
fetching companies in sector : banks-public-sector
14
10 -  15
fetching companies in sector : bearings
6
fetching companies in sector : breweries-distilleries
11
fetching companies in sector : cables-powerothers
0
sorry companies link is not present
fetching companies in sector : cables-telephone
10
fetching companies in sector : castings-forgings
15
15 -  20
fetching companies in sector : cement-major
22
fetching companies in sector : cement-mini
5
fetching companies in sector : cement-productsbuilding-materials

In [57]:
pd.concat(big_data).to_csv("0-50sector.csv")

In [59]:
for keys,values in big_data.items():
    values.to_csv("sector-wise data/"+keys+".csv")

In [47]:
big_data["banks-private-sector"]

,name,catagory,symbol,linkQuate,BS_standalone_link,EQUITY SHARE CAPITAL,TOTAL SHARE CAPITAL,RESERVES AND SURPLUS,TOTAL RESERVES AND SURPLUS,TOTAL SHAREHOLDERS FUNDS,...,NUMBER OF EMPLOYEES,CAPITAL ADEQUACY RATIOS (%),TIER 1 (%),TIER 2 (%),GROSS NPA,GROSS NPA (%),NET NPA,NET NPA (%),NET NPA TO ADVANCES (%),BILLS FOR COLLECTION
0,AU Small Financ,banksprivatesector,AUSMALLFINANC,ASF02,http://www.moneycontrol.com/financials/ausmall...,0.28,0.28,1.72,1.72,2.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Axis Bank,banksprivatesector,AXISBANK,AB16,https://www.moneycontrol.com/financials/axisba...,564.34,564.34,84383.51,84383.51,84947.84,...,74140.0,18.00,14.00,3.00,30233.82,5.0,20802.68,1.56,2.0,47842.76
2,Bandhan Bank,banksprivatesector,BANDHANBANK,BB09,https://www.moneycontrol.com/financials/bandha...,1610.25,1610.25,13585.21,13585.21,15195.46,...,39750.0,27.00,25.00,2.00,992.78,1.0,389.40,0.58,1.0,0.00
3,City Union Bank,banksprivatesector,CITYUNIONBANK,CUB,http://www.moneycontrol.com/financials/cityuni...,73.73,73.73,5222.34,5222.34,5296.07,...,5741.0,17.00,16.00,1.00,1413.40,4.0,778.49,2.29,2.0,335.79
4,DCB Bank,banksprivatesector,DCBBANK,DCB01,http://www.moneycontrol.com/financials/dcbbank...,310.42,310.42,2873.37,3110.76,3422.15,...,6845.0,17.75,13.90,3.85,631.51,2.0,293.51,1.16,1.0,431.99
5,Dhanlaxmi Bank,banksprivatesector,DHANLAXMIBANK,DB01,http://www.moneycontrol.com/financials/dhanlax...,253.01,253.01,434.75,573.90,826.91,...,1714.0,14.41,10.69,3.72,401.22,6.0,100.94,1.55,2.0,310.77
6,Federal Bank,banksprivatesector,FEDERALBANK,FB,https://www.moneycontrol.com/financials/federa...,398.53,398.53,14114.07,14119.08,14517.61,...,12496.0,14.00,13.00,1.00,3530.83,3.0,1607.17,1.31,1.0,3767.65
7,HDFC Bank,banksprivatesector,HDFCBANK,HDF01,https://www.moneycontrol.com/financials/hdfcba...,548.33,548.33,170437.70,170437.70,170986.03,...,116971.0,19.00,17.00,1.00,12649.97,1.0,3542.36,0.36,0.0,51584.90
8,ICICI Bank,banksprivatesector,ICICIBANK,ICI02,http://www.moneycontrol.com/financials/iciciba...,1294.76,1294.76,112091.29,115206.16,116504.41,...,99319.0,16.00,15.00,1.00,40829.09,6.0,9923.24,1.54,2.0,48216.24
9,IDFC First Bank,banksprivatesector,IDFCFIRSTBANK,IDF01,https://www.moneycontrol.com/financials/idfcfi...,4809.90,4809.90,10532.70,10532.70,15342.60,...,20222.0,13.38,13.03,0.08,2279.56,3.0,808.57,0.94,1.0,914.95


In [70]:
banking = []
for keys,values in big_data.items():
    if(keys == "banks-private-sector" or keys == "banks-public-sector"):
        banking.append(values)

In [73]:
# pd.concat(banking).to_csv("non_banking.csv")
pd.concat(banking).to_csv("banking.csv")

In [13]:
def company_anchor_to_balanceSheet(a):
    complink = "https://www.moneycontrol.com/"+a.attrs['href']
    i = a.attrs['href'].split('/')
    bslink = coy_to_BSlink(a,i[4],i[5])
    print(bslink)
    return soup_to_balanceSheet(bs(requests.get(bslink).text))